# Predicción de demanda de pasajeros

Este cuaderno está diseñado para predecir la demanda de pasajeros en rutas de transporte público utilizando **Facebook Prophet**.

**Estructura del cuaderno:**
1. [Importación de librerías y configuración](#1-importación-de-librerías-y-configuración)
2. [Configuración de parámetros](#2-configuración-de-parámetros)
3. [Carga y preprocesamiento de datos](#3-carga-y-preprocesamiento-de-datos)
4. [Entrenamiento de modelos](#4-entrenamiento-de-modelos-prophet)
5. [Generación de pronósticos](#5-entrenamiento-de-múltiples-modelos-y-generación-de-pronósticos)
6. [Visualizaciones de interes](#6-visualizaciones-de-interes)
7. [Descomposición y comparación de demanda real vs predicha](#7-descomposición-y-comparación-de-demanda-real-vs-predicha)
8. [Resumen de métricas](#8-resumen-de-métricas)
9. [Dudas frecuentes](#8-Dudas-frecuentes)

**Manual de uso para operadores**

El cuaderno está dividido en secciones (Celdas). Se recomienda ejecutar en orden desde la parte superior hasta el final, atendiendo las indicaciones en cada bloque de código. El flujo de trabajo típico es:

1. **Configurar parámetros:**
   - **`ruta_archivo`:** Ruta hacia el CSV con los datos históricos de demanda. Debe contener las columnas: `linea`, `fecha`, `vendeHora`, `sentido`.
   - **`split_date`:** Fecha de corte para separar conjunto de entrenamiento y prueba (por ejemplo, `'2024-06-01'`).
   - **`fecha_prediccion`:** Fecha a partir de la cual se generarán los pronósticos (ej. `'2025-04-08'`).
   - **`hora_inicio_operacion` y `hora_fin_operacion`:** Horas que definen el horario de operación (5 a 21).
   - **`frecuencia`:** Frecuencia de agregación temporal (ej. `'10min'`).
   - **`force_retrain`:** Si se coloca en `True`, se fuerza el reentrenamiento de modelos aunque existan ya guardados.

2. **Ejecutar celdas en orden:**
   1. Montaje de Google Drive. Aceptar permisos para conectar Google Drive. Asegurarse de tener las carpetas adecuadas para guardar los modelos y resultados.
   2. **`cargar_y_preparar_datos(ruta_archivo)`:** Cargará y limpiará los datos.
   3. **Entrenar o cargar modelos**: Ejecución de las celdas que llaman a `entrenar_y_guardar_modelos`.  
      - Esto entrenará los modelos Prophet para cada ruta y sentido detectados en el DataFrame (o cargará versiones previas si ya existen y `force_retrain=False`).  
   4. **Generar pronósticos**: Se utilizan los modelos entrenados para generar predicciones a partir de la fecha definida en `fecha_prediccion`.
   5. **Visualizaciones y análisis**: Para ver las gráficas de pronóstico (real vs. predicho, descomposición, métricas).

3. **Interpretar resultados:**
   - Se generará un archivo JSON (`metricas.json`) con las métricas de cada modelo, y un CSV con los pronósticos consolidados para cada ruta y sentido.  
   - El cuaderno mostrará interactivamente los gráficos de demanda real vs predicha, así como la descomposición del modelo Prophet.

4. **Dónde se guardan los archivos:**
   - **Modelos Entrenados**: Dentro de la carpeta `modelos_prophet/modelos/` en tu Google Drive (o la ruta que configures). Contiene archivos .joblib (uno por cada ruta y sentido).
   - **Pronósticos**: En `modelos_prophet/pronosticos/`. Aquí se guardan los CSV con predicciones futuras.
   - **Métricas**: En `modelos_prophet/metricas/`. Contiene el archivo metricas.json con RMSE y MAE de cada modelo.

5. **Recomendaciones adicionales:**
   - Verificar que el CSV de datos esté correctamente formateado antes de ejecutar el cuaderno.
   - Si los pronósticos parecen poco confiables o si tiene un nuevo set de historicos se puede forzar el reentrenamiento (`force_retrain=True`) o ajustar manualmente el rango de hiperparámetros en la celda de validación cruzada.
   - Cada vez que cambie los parámetros de entrenamiento, se sugiere respaldar los modelos antiguos para comparar su desempeño.



## 1. Herramientas utilizadas

Se importan todas las librerías requeridas para la ejecución del proyecto. Además, se configura el sistema de `logging` para registrar información crítica durante el entrenamiento y la generación de predicciones.


- **pandas** y **numpy**: librerías base para la manipulación de datos.
- **prophet**: librería de modelado de series temporales desarrollada por Facebook/Meta.
- **matplotlib** y **plotly**: para generar gráficas y visualizaciones (estáticas e interactivas).
- **joblib**: para la serialización y deserialización (guardar/cargar) de los modelos entrenados.
- **logging**: para manejar mensajes informativos, advertencias y errores, permitiendo un mejor seguimiento de la ejecución.
- **os**: para la gestión de rutas y archivos en el sistema.
- **datetime**: para manejar fechas y tiempos de manera adecuada.
- **json**: para almacenar o leer información de métricas y configuraciones.


In [ ]:
# =========================================
# 1. Importación de Librerías y Configuración
# =========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.make_holidays import make_holidays_df
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os
from datetime import datetime
import logging
import json


# Configuración avanzada de logging
logger = logging.getLogger('ProphetLogger')
logger.setLevel(logging.INFO)

# Crear handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('prophet_model.log')
c_handler.setLevel(logging.INFO)
f_handler.setLevel(logging.INFO)

# Crear formato y añadir a handlers
c_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Añadir handlers al logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

logger.info("Inicio del cuaderno de predicción de demanda de buses.")


2025-01-04 04:41:45,266 - INFO - Inicio del cuaderno de predicción de demanda de buses.
INFO:ProphetLogger:Inicio del cuaderno de predicción de demanda de buses.


## 2. Configuración de parámetros

Los operadores pueden ajustar los parámetros clave del modelo según sus las caracteristicas de la operación. Esto incluye la ruta del archivo de datos, fechas de división y predicción, horario de operación y frecuencia de agregación de datos.

**Parámetros configurables:**
- **Ruta del archivo de datos (`ruta_archivo`):** ruta al archivo CSV con los datos históricos.
- **Fecha de división (`split_date`):** fecha para dividir los datos en conjuntos de entrenamiento y prueba.
- **Fecha de predicción (`fecha_prediccion`):** fecha de inicio para generar pronósticos futuros.
- **Horario de operación (`hora_inicio_operacion` y `hora_fin_operacion`):** hora de inicio y fin de operación.
- **Frecuencia de agregación (`frecuencia`):** intervalo de tiempo para la agregación de datos (ej. '5min', '10min', '15min').


**Estructura de Directorios en Google Drive:**
- `modelos_prophet/modelos/`: Aquí se guardan los archivos de los modelos entrenados (`.joblib`).
- `modelos_prophet/pronosticos/`: Aquí se guardan los archivos CSV con los pronósticos generados.
- `modelos_prophet/metricas/`: Aquí se almacenan las métricas de desempeño del modelo (`metricas.json`), así como archivos adicionales si fuera necesario.

> **Nota:** En caso de necesitar más flexibilidad, puedes modificar la ruta base `/content/drive/MyDrive/Colab Notebooks/modelos_prophet/` a la ubicación que prefieras en Google Drive o en tu entorno local.


In [ ]:
# =========================================
# 2. Configuración de Parámetros Interactiva
# =========================================

# Montaje de Google Drive (si se usa Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Parámetros globales
ruta_archivo = "/content/drive/MyDrive/Colab Notebooks/info.csv"
split_date = '2024-06-01'
fecha_prediccion = '2025-04-08'
hora_inicio_operacion = 5  # 5 am
hora_fin_operacion = 21 # 9 pm
frecuencia = '10min'  # Intervalo de 10 minutos
force_retrain=False

# Definir subdirectorios en Google Drive (o localmente)
modelos_subdir = os.path.join('/content/drive/MyDrive/Colab Notebooks/modelos_prophet/', 'modelos')
pronosticos_subdir = os.path.join('/content/drive/MyDrive/Colab Notebooks/modelos_prophet/', 'pronosticos')
metricas_subdir = os.path.join('/content/drive/MyDrive/Colab Notebooks/modelos_prophet/', 'metricas')

# Crear los subdirectorios si no existen
for subdir in [modelos_subdir, pronosticos_subdir, metricas_subdir]:
    os.makedirs(subdir, exist_ok=True)

Mounted at /content/drive


## 3. Carga y preprocesamiento de datos

Se carga el archivo CSV especificado, se verifica su existencia y se generan las columnas derivadas necesarias para el modelado. Además, se mapean las líneas de transporte a nombres de rutas más descriptivos para una mejor interpretación.

1. **Carga de datos:** importación del archivo CSV y selección de columnas relevantes.
2. **Conversión de tipos:** aseguramiento de los tipos de datos adecuados para cada columna.
3. **Generación de timestamps:** combinación de la fecha y la hora para crear una columna de datetime.
4. **Creación de variables derivadas:** generación de variables como día de la semana, mes, hora en formato entero, indicador de fin de semana y hora pico.
5. **Mapeo de rutas:** asignación de nombres descriptivos a las rutas basados en los números de línea.


In [ ]:
# =========================================
# 3. Carga y Preprocesamiento de Datos
# =========================================

def cargar_y_preparar_datos(ruta_archivo):
    """
    Carga y preprocesa los datos desde el archivo CSV.

    Parámetros:
    - ruta_archivo (str): Ruta al archivo CSV con los datos históricos.

    Retorna:
    - data (pd.DataFrame): Datos preprocesados listos para el modelado.

    Excepciones:
    - FileNotFoundError: Si el archivo especificado no existe.
    - Otros errores de procesamiento de datos.
    """
    try:
        # Verificación de la existencia del archivo
        if not os.path.isfile(ruta_archivo):
            logger.error(f"El archivo {ruta_archivo} no existe.")
            raise FileNotFoundError(f"El archivo {ruta_archivo} no existe.")

        # Cargar los datos
        data = pd.read_csv(ruta_archivo, sep=';')
        # Seleccionar columnas relevantes y eliminar filas con valores nulos
        data = data[['linea', 'fecha', 'vendeHora', 'sentido']].dropna()
        # Convertir tipos de datos
        data = data.astype({'linea':'int64','sentido':'int64'})
        # Convertir fecha y hora a datetime
        data['fecha'] = pd.to_datetime(data['fecha'], format='%Y%m%d')
        data['vendeHora'] = data['vendeHora'].astype(str).str.zfill(4)
        data['hora'] = pd.to_datetime(data['vendeHora'], format='%H%M').dt.time
        data['datetime'] = pd.to_datetime(data.apply(lambda row: datetime.combine(row['fecha'], row['hora']), axis=1))

        # Generar columnas derivadas
        data['dia_semana'] = data['fecha'].dt.day_name()
        data['mes'] = data['fecha'].dt.month_name()
        data['hora_int'] = data['hora'].apply(lambda x: x.hour)
        data['es_fin_de_semana'] = data['dia_semana'].isin(['Saturday', 'Sunday']).astype(int)
        data['es_hora_pico'] = data['hora_int'].isin([7,8,9,17,18,19]).astype(int)

        # Mapeo de líneas a rutas
        linea_a_ruta = {
            1: 'Especial', 2: 'Pradera UPB FS', 3: 'PANZENU', 5: 'DORADO', 6: 'KM30',
            7: 'SANTANDER', 8: 'TAMBO CIRCUNVALAR', 9: 'MOGAMBO 22', 12: '6 DE MARZO',
            13: 'CARRIZAL', 17: 'PRADERA 27', 18: 'Furatena', 20: 'KM 15', 21: 'SANTA LUCIA',
            24: 'TERMINAL - AEROPUERTO', 26: 'RANCHO GRANDE', 27: 'SABANAL', 34: 'LETICIA',
            36: 'AGUAS VIVAS', 37: 'CAÑO VIEJO', 97: 'Auditorias RE'
        }
        data['ruta'] = data['linea'].map(linea_a_ruta)

        logger.info("Datos cargados y preprocesados exitosamente.")
        return data

    except Exception as e:
        logger.error(f"Error en cargar_y_preparar_datos: {e}")
        raise

# Cargar datos
data = cargar_y_preparar_datos(ruta_archivo)
rutas = data['ruta'].unique().tolist()
sentidos = [1, 2]


<ipython-input-3-6fd4f7bd8dc6>:26: DtypeWarning: Columns (7,21,33) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(ruta_archivo, sep=';')
2025-01-04 04:43:35,326 - INFO - Datos cargados y preprocesados exitosamente.
INFO:ProphetLogger:Datos cargados y preprocesados exitosamente.


## 4. Entrenamiento de modelos

Se entrena (o carga) el modelo Prophet para cada ruta y sentido. Se implementa una lógica que permite:

- **Verificar la existencia del modelo:** si el modelo ya existe en Google Drive y no se fuerza el reentrenamiento, se carga el modelo existente.
- **Filtrar datos:** Filtrar los datos según la ruta y sentido específicos.
- **Validación cruzada condicional:** realizar validación cruzada si hay suficientes datos para optimizar los hiperparámetros; de lo contrario, usar parámetros por defecto.
- **Entrenamiento final del modelo:** entrenar el modelo con los mejores hiperparámetros obtenidos o con los parámetros por defecto.
- **Evaluación y guardado del modelo:** evaluar el modelo en el conjunto de prueba y guardarlo en Google Drive.
- **Versionado de modelos:** incorporar la fecha de entrenamiento en el nombre del archivo del modelo para facilitar el versionado.
- **Gestión de métricas:** mantener un registro de las métricas de desempeño (RMSE, MAE) en un archivo JSON sincronizado con los modelos.

**Nota:** Todos los modelos, pronósticos y métricas se almacenan en subdirectorios específicos dentro de Google Drive para mantener una organización clara.


In [ ]:
# =========================================
# 4. Entrenamiento de Modelos Prophet
# =========================================

def guardar_metricas(metricas, metricas_subdir):
    """
    Guarda las métricas en un archivo JSON.
    """
    metricas_path = os.path.join(metricas_subdir, 'metricas.json')
    try:
        with open(metricas_path, 'w') as f:
            json.dump(metricas, f, indent=4)
        logger.info("Métricas guardadas exitosamente en 'metricas.json'.")
    except Exception as e:
        logger.error(f"Error al guardar métricas: {e}")

def cargar_metricas(metricas_subdir):
    """
    Carga las métricas desde un archivo JSON.
    """
    metricas_path = os.path.join(metricas_subdir, 'metricas.json')
    if os.path.isfile(metricas_path):
        try:
            with open(metricas_path, 'r') as f:
                metricas = json.load(f)
            logger.info("Métricas cargadas exitosamente desde 'metricas.json'.")
            return metricas
        except Exception as e:
            logger.error(f"Error al cargar métricas: {e}")
            return {}
    else:
        logger.warning("No se encontró 'metricas.json'. Se creará uno nuevo al guardar métricas.")
        return {}

def entrenar_modelo_prophet(
    data,
    ruta,
    sentido,
    frecuencia='10min',
    split_date='2024-06-01',
    fecha_prediccion='2025-04-08',
    hora_inicio_operacion=5,
    hora_fin_operacion=21,
    modelos_subdir=None,
    metricas=None,
    force_retrain=False
):
    """
    Entrena un modelo Prophet para una ruta y sentido específicos, o carga el modelo existente.
    También carga las métricas y datos de prueba si el modelo ya existe.

    Parámetros:
    - data (pd.DataFrame): Datos preprocesados.
    - ruta (str): Nombre de la ruta.
    - sentido (int): Sentido de la ruta (1 o 2).
    - frecuencia (str): Frecuencia de agregación temporal (ej. '10min').
    - split_date (str): Fecha para dividir los datos en entrenamiento y prueba.
    - fecha_prediccion (str): Fecha de inicio para pronósticos futuros.
    - hora_inicio_operacion (int): Hora de inicio de operación.
    - hora_fin_operacion (int): Hora de fin de operación.
    - modelos_subdir (str): Directorio para guardar/cargar modelos.
    - metricas (dict): Diccionario de métricas cargadas previamente.
    - force_retrain (bool): Si True, fuerza el reentrenamiento del modelo.

    Retorna:
    - model (Prophet): Modelo entrenado o cargado.
    - train_prophet (pd.DataFrame): Conjunto de entrenamiento (None si se carga modelo).
    - test_prophet (pd.DataFrame): Conjunto de prueba (cargado si modelo existe).
    - metrics (dict): Métricas (RMSE, MAE).
    """
    clave = f"{ruta}_sentido_{sentido}"
    model_filename = f"{clave}_freq_{frecuencia}.joblib"
    model_path = os.path.join(modelos_subdir, model_filename)
    test_filename = f"{clave}_test.csv"
    test_path = os.path.join(modelos_subdir, test_filename)

    # Verificar si el modelo ya existe
    if not force_retrain and os.path.isfile(model_path):
        logger.info(f"Cargando modelo existente para {ruta} sentido {sentido} desde {model_filename}.")
        model = joblib.load(model_path)

        # Cargar datos de prueba
        if os.path.isfile(test_path):
            test_prophet = pd.read_csv(test_path, parse_dates=['ds'])
            logger.info(f"Datos de prueba cargados desde {test_filename}.")
        else:
            logger.warning(f"No se encontró el archivo de datos de prueba para {clave}.")
            test_prophet = None

        # Obtener métricas desde el diccionario de métricas
        metrics = metricas.get(clave, {'RMSE': None, 'MAE': None})
        return model, None, test_prophet, metrics

    # Filtrar datos por ruta y sentido
    data_filtrada = data[(data['ruta'] == ruta) & (data['sentido'] == sentido)]
    if data_filtrada.empty:
        logger.warning(f"No hay datos para la ruta {ruta} en sentido {sentido}.")
        return None, None, None, None

    # Resamplear datos
    data_filtrada = data_filtrada.set_index('datetime')
    data_resampled = data_filtrada.resample(frecuencia).size().reset_index(name='demanda')

    # Generar columnas derivadas
    data_resampled['dia_semana'] = data_resampled['datetime'].dt.day_name()
    data_resampled['hora'] = data_resampled['datetime'].dt.hour
    data_resampled['es_fin_de_semana'] = data_resampled['dia_semana'].isin(['Saturday', 'Sunday']).astype(int)
    data_resampled['es_hora_pico'] = data_resampled['hora'].isin([7,8,9,17,18,19]).astype(int)

    # Definir feriados específicos de Colombia
    colombia_holidays = pd.DataFrame({
        'holiday': 'colombia_holiday',
        'ds': pd.to_datetime([
            '2024-01-01', '2024-03-20', '2024-05-01', '2024-07-20',
            '2024-08-07', '2024-10-16', '2024-11-06', '2024-12-08',
            '2024-12-25'
        ]),
        'lower_window': 0,
        'upper_window': 1,
    })

    # Preparar datos para Prophet
    data_prophet = data_resampled.rename(columns={'datetime': 'ds', 'demanda': 'y'})
    data_prophet['es_hora_pico'] = data_resampled['es_hora_pico']
    data_prophet['es_fin_de_semana'] = data_resampled['es_fin_de_semana']
    data_prophet['es_dia_festivo'] = data_prophet['ds'].isin(colombia_holidays['ds']).astype(int)

    # Dividir en entrenamiento y prueba
    train = data_prophet[data_prophet['ds'] <= split_date].copy()
    test = data_prophet[data_prophet['ds'] > split_date].copy()

    if train.empty:
        logger.warning(f"Entrenamiento vacío para {ruta}, sentido {sentido}.")
        return None, None, None, None
    if test.empty:
        logger.warning(f"Prueba vacía para {ruta}, sentido {sentido}.")
        return None, None, None, None

    # Parámetros para validación cruzada
    initial = '180 days'  # Cantidad de datos inicial para entrenar
    horizon = '30 days'   # Horizonte de predicción
    period = '30 days'    # Periodo entre validaciones

    # Convertir period strings a días
    def periodo_en_dias(period_str):
        return int(period_str.split()[0])

    dias_initial = periodo_en_dias(initial)
    dias_horizon = periodo_en_dias(horizon)

    # Calcular duración total del entrenamiento
    total_days = (train['ds'].max() - train['ds'].min()).days

    # Decidir si realizar cross_validation
    usar_cross_val = (total_days > (dias_initial + dias_horizon))

    if usar_cross_val:
        logger.info(f"Realizando validación cruzada para {ruta} sentido {sentido}.")
        param_grid = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]
        }

        best_rmse = float('inf')
        best_params = {}

        for cps in param_grid['changepoint_prior_scale']:
            for sps in param_grid['seasonality_prior_scale']:
                model_temp = Prophet(
                    holidays=colombia_holidays,
                    changepoint_prior_scale=cps,
                    seasonality_prior_scale=sps
                )
                model_temp.add_regressor('es_hora_pico')
                model_temp.add_regressor('es_fin_de_semana')
                model_temp.add_regressor('es_dia_festivo')
                model_temp.fit(train)

                try:
                    df_cv = cross_validation(
                        model_temp,
                        initial=initial,
                        period=period,
                        horizon=horizon,
                        parallel="processes"
                    )
                    df_p = performance_metrics(df_cv, rolling_window=1)
                    rmse = df_p['rmse'].mean()
                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_params = {'changepoint_prior_scale': cps, 'seasonality_prior_scale': sps}
                except ValueError as ve:
                    logger.warning(f"Cross-validation fallida para {ruta} sentido {sentido} con cps={cps}, sps={sps}: {ve}")
                    continue

        if not best_params:
            best_params = {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 1.0}
            logger.info(f"No se pudo realizar cross_validation adecuadamente para {ruta} sentido {sentido}, usando parámetros por defecto.")
        else:
            logger.info(f"Mejores hiperparámetros para {ruta} sentido {sentido}: {best_params}")
            logger.info(f"RMSE validación cruzada: {best_rmse}")
    else:
        best_params = {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 1.0}
        logger.info(f"No hay suficientes datos para cross_validation en {ruta} sentido {sentido}, usando parámetros por defecto: {best_params}")

    # Entrenar modelo final con los parámetros elegidos
    model = Prophet(
        holidays=colombia_holidays,
        changepoint_prior_scale=best_params['changepoint_prior_scale'],
        seasonality_prior_scale=best_params['seasonality_prior_scale']
    )
    model.add_regressor('es_hora_pico')
    model.add_regressor('es_fin_de_semana')
    model.add_regressor('es_dia_festivo')
    model.fit(train)

    # Evaluación en prueba
    forecast = model.predict(test)
    rmse_test = np.sqrt(mean_squared_error(test['y'], forecast['yhat']))
    mae_test = mean_absolute_error(test['y'], forecast['yhat'])
    metrics = {'RMSE': rmse_test, 'MAE': mae_test}
    logger.info(f"Evaluación en prueba para {ruta} sentido {sentido}: RMSE = {rmse_test}, MAE = {mae_test}")

    # Guardar modelo en subdirectorio 'modelos'
    joblib.dump(model, model_path)
    logger.info(f"Modelo guardado en {model_path}")

    # Guardar datos de prueba
    test.to_csv(test_path, index=False)
    logger.info(f"Datos de prueba guardados en {test_path}")

    # Actualizar métricas
    clave = f"{ruta}_sentido_{sentido}"
    metricas[clave] = metrics

    return model, train, test, metrics


def entrenar_y_guardar_modelos(data, rutas, sentidos, parametros, modelos_subdir, metricas, force_retrain):
    """
    Entrena o carga modelos Prophet para cada combinación de ruta y sentido.
    Actualiza las métricas y carga los datos de prueba si es necesario.

    Parámetros:
    - data (pd.DataFrame): Datos preprocesados.
    - rutas (list): Lista de rutas.
    - sentidos (list): Lista de sentidos.
    - parametros (dict): Diccionario con parámetros de configuración.
    - modelos_subdir (str): Directorio para guardar/cargar modelos.
    - metricas (dict): Diccionario de métricas cargadas previamente.
    - force_retrain (bool): Si True, fuerza el reentrenamiento de todos los modelos.

    Retorna:
    - modelos (dict): Diccionario de modelos entrenados o cargados.
    - train_data (dict): Diccionario de conjuntos de entrenamiento (None si modelo cargado).
    - test_data (dict): Diccionario de conjuntos de prueba.
    - metricas_actualizadas (dict): Diccionario de métricas actualizadas.
    """
    modelos = {}
    train_data = {}
    test_data = {}
    metricas_actualizadas = metricas.copy()

    for ruta in rutas:
        for sentido in sentidos:
            clave = f"{ruta}_sentido_{sentido}"
            model, train_prophet, test_prophet, metrics = entrenar_modelo_prophet(
                data=data,
                ruta=ruta,
                sentido=sentido,
                frecuencia=parametros['frecuencia'],
                split_date=parametros['split_date'],
                fecha_prediccion=parametros['fecha_prediccion'],
                hora_inicio_operacion=parametros['hora_inicio_operacion'],
                hora_fin_operacion=parametros['hora_fin_operacion'],
                modelos_subdir=modelos_subdir,
                metricas=metricas,
                force_retrain=force_retrain
            )
            if model is not None:
                modelos[clave] = model
                if metrics:
                    metricas_actualizadas[clave] = metrics
                else:
                    # Obtener métricas existentes
                    metricas_actualizadas[clave] = metricas.get(clave, {'RMSE': None, 'MAE': None})

                if test_prophet is not None:
                    test_data[clave] = test_prophet

                # Registrar y mostrar información
                if metrics:
                    logger.info(f"Modelo listo para {clave}. RMSE: {metrics['RMSE']}, MAE: {metrics['MAE']}")
                    print(f"Modelo listo para {clave}.")
                else:
                    logger.info(f"Modelo cargado para {clave}. Métricas existentes.")
                    print(f"Modelo cargado para {clave}.")
            else:
                logger.warning(f"No se entrenó modelo para {ruta} sentido {sentido}.")
                print(f"No se entrenó modelo para {ruta} sentido {sentido}.")

    return modelos, train_data, test_data, metricas_actualizadas


In [ ]:
# Cargar métricas existentes
metricas = cargar_metricas(metricas_subdir)

# Definir parámetros
parametros = {
    'frecuencia': frecuencia,
    'split_date': split_date,
    'fecha_prediccion': fecha_prediccion,
    'hora_inicio_operacion': hora_inicio_operacion,
    'hora_fin_operacion': hora_fin_operacion
}

# Entrenar o cargar modelos
modelos, train_data, test_data, metricas_actualizadas = entrenar_y_guardar_modelos(
    data=data,
    rutas=rutas,
    sentidos=sentidos,
    parametros=parametros,
    modelos_subdir=modelos_subdir,
    metricas=metricas,
    force_retrain=force_retrain
)

# Guardar métricas actualizadas
guardar_metricas(metricas_actualizadas, metricas_subdir)


2025-01-04 04:43:36,358 - INFO - Métricas cargadas exitosamente desde 'metricas.json'.
INFO:ProphetLogger:Métricas cargadas exitosamente desde 'metricas.json'.
2025-01-04 04:43:36,364 - INFO - Cargando modelo existente para SANTANDER sentido 1 desde SANTANDER_sentido_1_freq_10min.joblib.
INFO:ProphetLogger:Cargando modelo existente para SANTANDER sentido 1 desde SANTANDER_sentido_1_freq_10min.joblib.
2025-01-04 04:43:38,707 - INFO - Datos de prueba cargados desde SANTANDER_sentido_1_test.csv.
INFO:ProphetLogger:Datos de prueba cargados desde SANTANDER_sentido_1_test.csv.
2025-01-04 04:43:38,711 - INFO - Modelo listo para SANTANDER_sentido_1. RMSE: 9.209471183476026, MAE: 7.667959624723528
INFO:ProphetLogger:Modelo listo para SANTANDER_sentido_1. RMSE: 9.209471183476026, MAE: 7.667959624723528
2025-01-04 04:43:38,718 - INFO - Cargando modelo existente para SANTANDER sentido 2 desde SANTANDER_sentido_2_freq_10min.joblib.
INFO:ProphetLogger:Cargando modelo existente para SANTANDER sentido

Modelo listo para SANTANDER_sentido_1.


2025-01-04 04:43:40,088 - INFO - Datos de prueba cargados desde SANTANDER_sentido_2_test.csv.
INFO:ProphetLogger:Datos de prueba cargados desde SANTANDER_sentido_2_test.csv.
2025-01-04 04:43:40,094 - INFO - Modelo listo para SANTANDER_sentido_2. RMSE: 0.7860467454984671, MAE: 0.23322059004966172
INFO:ProphetLogger:Modelo listo para SANTANDER_sentido_2. RMSE: 0.7860467454984671, MAE: 0.23322059004966172
2025-01-04 04:43:40,101 - INFO - Cargando modelo existente para AGUAS VIVAS sentido 1 desde AGUAS VIVAS_sentido_1_freq_10min.joblib.
INFO:ProphetLogger:Cargando modelo existente para AGUAS VIVAS sentido 1 desde AGUAS VIVAS_sentido_1_freq_10min.joblib.


Modelo listo para SANTANDER_sentido_2.


2025-01-04 04:43:41,647 - INFO - Datos de prueba cargados desde AGUAS VIVAS_sentido_1_test.csv.
INFO:ProphetLogger:Datos de prueba cargados desde AGUAS VIVAS_sentido_1_test.csv.
2025-01-04 04:43:41,651 - INFO - Modelo listo para AGUAS VIVAS_sentido_1. RMSE: 4.3242584942301665, MAE: 2.6930021459572386
INFO:ProphetLogger:Modelo listo para AGUAS VIVAS_sentido_1. RMSE: 4.3242584942301665, MAE: 2.6930021459572386
2025-01-04 04:43:41,657 - INFO - Cargando modelo existente para AGUAS VIVAS sentido 2 desde AGUAS VIVAS_sentido_2_freq_10min.joblib.
INFO:ProphetLogger:Cargando modelo existente para AGUAS VIVAS sentido 2 desde AGUAS VIVAS_sentido_2_freq_10min.joblib.


...

Modelo listo para TERMINAL - AEROPUERTO_sentido_1.


2025-01-04 04:44:32,074 - INFO - Datos de prueba cargados desde TERMINAL - AEROPUERTO_sentido_2_test.csv.
INFO:ProphetLogger:Datos de prueba cargados desde TERMINAL - AEROPUERTO_sentido_2_test.csv.
2025-01-04 04:44:32,079 - INFO - Modelo listo para TERMINAL - AEROPUERTO_sentido_2. RMSE: 0.1102806422526531, MAE: 0.012191475752564802
INFO:ProphetLogger:Modelo listo para TERMINAL - AEROPUERTO_sentido_2. RMSE: 0.1102806422526531, MAE: 0.012191475752564802
2025-01-04 04:44:32,085 - INFO - Cargando modelo existente para SABANAL sentido 1 desde SABANAL_sentido_1_freq_10min.joblib.
INFO:ProphetLogger:Cargando modelo existente para SABANAL sentido 1 desde SABANAL_sentido_1_freq_10min.joblib.


Modelo listo para TERMINAL - AEROPUERTO_sentido_2.


2025-01-04 04:44:33,609 - INFO - Datos de prueba cargados desde SABANAL_sentido_1_test.csv.
INFO:ProphetLogger:Datos de prueba cargados desde SABANAL_sentido_1_test.csv.
2025-01-04 04:44:33,617 - INFO - Modelo listo para SABANAL_sentido_1. RMSE: 0.17730435835742167, MAE: 0.04111508624983991
INFO:ProphetLogger:Modelo listo para SABANAL_sentido_1. RMSE: 0.17730435835742167, MAE: 0.04111508624983991
2025-01-04 04:44:33,926 - WARNING - Prueba vacía para SABANAL, sentido 2.
2025-01-04 04:44:33,930 - WARNING - No se entrenó modelo para SABANAL sentido 2.
2025-01-04 04:44:33,946 - INFO - Métricas guardadas exitosamente en 'metricas.json'.
INFO:ProphetLogger:Métricas guardadas exitosamente en 'metricas.json'.


Modelo listo para SABANAL_sentido_1.
No se entrenó modelo para SABANAL sentido 2.


## 5. Generación de pronósticos

Se generan pronósticos futuros utilizando los modelos entrenados para la fecha y configuración especificadas. Los resultados se visualiza y se consolidan en un único archivo CSV para facilitar su análisis.

1. **Generación de pronósticos:** Para cada modelo, se generan pronósticos futuros en el horario definido.
2. **Visualización de pronósticos:** Se visualizan los pronósticos.
3. **Guardado de pronósticos:** Los pronósticos generados se consolidan en un único archivo CSV para facilitar su análisis posterior.


In [ ]:
# =========================================
# 5. Entrenamiento de Múltiples Modelos y Generación de Pronósticos
# =========================================

def calcular_periodos(hora_inicio, hora_fin, frecuencia):
    """
    Calcula el número de periodos basados en el horario de operación y la frecuencia.

    Parámetros:
    - hora_inicio (int): Hora de inicio de operación.
    - hora_fin (int): Hora de fin de operación.
    - frecuencia (str): Frecuencia de agregación temporal (ej. '10min').

    Retorna:
    - periodos (int): Número de periodos.
    """
    minutos_por_periodo = int(frecuencia.replace('min', ''))  # Asumiendo que frecuencia está en formato 'min'
    total_minutos = (hora_fin - hora_inicio)*60
    return total_minutos // minutos_por_periodo

def generar_pronostico_futuro(model, fecha_prediccion, frecuencia='10min', hora_inicio=5, hora_fin=21):
    """
    Genera pronósticos futuros utilizando el modelo entrenado.

    Parámetros:
    - model (Prophet): Modelo entrenado.
    - fecha_prediccion (str): Fecha de inicio para el pronóstico (ej: '2025-04-08').
    - frecuencia (str): Frecuencia de los pronósticos (ej: '10min').
    - hora_inicio (int): Hora de inicio de operación.
    - hora_fin (int): Hora de fin de operación.

    Retorna:
    - forecast (pd.DataFrame): DataFrame con las predicciones.
    """
    periodos = calcular_periodos(hora_inicio, hora_fin, frecuencia)
    future = pd.DataFrame({'ds': pd.date_range(start=f"{fecha_prediccion} {hora_inicio:02d}:00:00", periods=periodos, freq=frecuencia)})

    año_prediccion = future['ds'].dt.year.unique()[0]
    colombia_holidays_future = make_holidays_df(year_list=[año_prediccion], country='CO')

    future['es_hora_pico'] = future['ds'].dt.hour.isin([7,8,9,17,18,19]).astype(int)
    future['es_fin_de_semana'] = future['ds'].dt.day_name().isin(['Saturday','Sunday']).astype(int)
    future['es_dia_festivo'] = future['ds'].isin(colombia_holidays_future['ds']).astype(int)

    forecast = model.predict(future)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

def obtener_demanda_predicha(forecast, clave, fecha_prediccion):
    """
    Grafica la demanda predicha de manera interactiva y retorna la demanda predicha.

    Parámetros:
    - forecast (pd.DataFrame): DataFrame con las predicciones.
    - clave (str): Clave del modelo (ruta y sentido).
    - fecha_prediccion (str): Fecha de inicio para el pronóstico.

    Retorna:
    - demanda_predicha (np.ndarray): Array con la demanda predicha.
    """
    demanda_predicha = np.maximum(forecast['yhat'].values.astype(int), 0)

    # Crear gráfica interactiva con Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast['ds'], y=demanda_predicha, mode='lines+markers', name='Demanda Predicha'))
    fig.update_layout(
        title=f'Demanda Predicha para {clave} - Fecha: {fecha_prediccion}',
        xaxis_title='Tiempo',
        yaxis_title='Demanda Predicha',
        hovermode='closest'
    )
    fig.show()

    return demanda_predicha

def generar_pronosticos_futuros(modelos, fecha_prediccion, frecuencia, hora_inicio, hora_fin, pronosticos_subdir):
    """
    Genera pronósticos futuros para todos los modelos entrenados y los guarda en un archivo CSV consolidado.

    Parámetros:
    - modelos (dict): Diccionario con los modelos entrenados.
    - fecha_prediccion (str): Fecha de inicio para el pronóstico (ej: '2025-04-08').
    - frecuencia (str): Frecuencia de los pronósticos (ej: '10min').
    - hora_inicio (int): Hora de inicio de operación.
    - hora_fin (int): Hora de fin de operación.
    - pronosticos_subdir (str): Directorio para guardar los pronósticos.

    Retorna:
    - demanda_predicha (dict): Diccionario con las demandas predichas.
    """
    demanda_predicha = {}

    for clave, model in modelos.items():
        try:
            forecast_future = generar_pronostico_futuro(
                model=model,
                fecha_prediccion=fecha_prediccion,
                frecuencia=frecuencia,
                hora_inicio=hora_inicio,
                hora_fin=hora_fin
            )
            demanda = obtener_demanda_predicha(forecast_future, clave, fecha_prediccion)
            demanda_predicha[clave] = demanda

            # Guardar pronóstico individual
            pronostico_filename = f"{clave}_pronostico_{fecha_prediccion}.csv"
            pronostico_path = os.path.join(pronosticos_subdir, pronostico_filename)
            forecast_future.to_csv(pronostico_path, index=False)
            logger.info(f"Pronóstico guardado en {pronostico_path}")
            print(f"Pronóstico generado y guardado para {clave}.")

        except Exception as e:
            logger.error(f"Error al generar pronóstico para {clave}: {e}")
            print(f"Error al generar pronóstico para {clave}: {e}")

    return demanda_predicha

# Generar pronósticos futuros
demanda_predicha = generar_pronosticos_futuros(
    modelos=modelos,
    fecha_prediccion=fecha_prediccion,
    frecuencia=frecuencia,
    hora_inicio=hora_inicio_operacion,
    hora_fin=hora_fin_operacion,
    pronosticos_subdir=pronosticos_subdir
)

# Guardar pronósticos en un archivo CSV consolidado
if demanda_predicha:
    df_pronosticos = pd.DataFrame(demanda_predicha)
    df_pronosticos.index = pd.to_datetime(pd.date_range(start=f"{fecha_prediccion} {hora_inicio_operacion:02d}:00:00",
                                                           periods=calcular_periodos(hora_inicio_operacion, hora_fin_operacion, frecuencia),
                                                           freq=frecuencia))
    df_pronosticos = df_pronosticos.transpose()
    pronosticos_consolidados_path = os.path.join(pronosticos_subdir, 'pronosticos_futuros_consolidados.csv')
    df_pronosticos.to_csv(pronosticos_consolidados_path)
    logger.info(f"Pronósticos consolidados guardados en '{pronosticos_consolidados_path}'.")
    print(f"Pronósticos consolidados guardados en '{pronosticos_consolidados_path}'.")
else:
    logger.info("No se generaron pronósticos.")
    print("No se generaron pronósticos.")


2025-01-04 04:44:35,503 - INFO - Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SANTANDER_sentido_1_pronostico_2025-04-08.csv
INFO:ProphetLogger:Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SANTANDER_sentido_1_pronostico_2025-04-08.csv


Pronóstico generado y guardado para SANTANDER_sentido_1.


2025-01-04 04:44:36,288 - INFO - Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SANTANDER_sentido_2_pronostico_2025-04-08.csv
INFO:ProphetLogger:Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SANTANDER_sentido_2_pronostico_2025-04-08.csv


Pronóstico generado y guardado para SANTANDER_sentido_2.


2025-01-04 04:44:37,096 - INFO - Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/AGUAS VIVAS_sentido_1_pronostico_2025-04-08.csv
INFO:ProphetLogger:Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/AGUAS VIVAS_sentido_1_pronostico_2025-04-08.csv


...

Pronóstico generado y guardado para TERMINAL - AEROPUERTO_sentido_2.


2025-01-04 04:45:04,876 - INFO - Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SABANAL_sentido_1_pronostico_2025-04-08.csv
INFO:ProphetLogger:Pronóstico guardado en /content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/SABANAL_sentido_1_pronostico_2025-04-08.csv


Pronóstico generado y guardado para SABANAL_sentido_1.


2025-01-04 04:45:05,533 - INFO - Pronósticos consolidados guardados en '/content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/pronosticos_futuros_consolidados.csv'.
INFO:ProphetLogger:Pronósticos consolidados guardados en '/content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/pronosticos_futuros_consolidados.csv'.


Pronósticos consolidados guardados en '/content/drive/MyDrive/Colab Notebooks/modelos_prophet/pronosticos/pronosticos_futuros_consolidados.csv'.


## 7. Comparación de demanda real vs predicha

Se selecciona una ruta y sentido específicos para ilustrar cómo descomponer el modelo y comparar los valores reales vs. predichos. Esta funcionalidad puede ser reutilizada para cualquier modelo entrenado, proporcionando una herramienta flexible para el análisis detallado de los modelos.

1. **Selección de Modelo de Ejemplo:** Se elige una combinación de ruta y sentido para el análisis.
2. **Comparación Real vs. Predicho**:
   - Se superponen los valores reales (`y`) con las predicciones (`yhat`) del modelo en el conjunto de prueba.
   - Es **fundamental** seleccionar un intervalo de tiempo donde **sí existan datos de prueba**.  
     - Por ejemplo, si `split_date = '2024-06-01'`, cualquier fecha posterior a esa (ej.: `'2024-06-15'`) puede servir para la comparación, siempre y cuando esté en el *conjunto de prueba*.
   - Asegúrate de que la fecha o rango de fechas elegido **corresponde** a datos que efectivamente no se hayan utilizado en el entrenamiento. De lo contrario, la comparación no será representativa.

3. **Interpretación**:
   - Si la curva de predicción se aleja mucho de la curva real, indica potenciales problemas de sobreajuste o subajuste, falta de datos, cambio de patrones, etc.
   - Observa si hay variaciones sistemáticas en cierto horario (ej. horas pico) o día de la semana que el modelo no está capturando.



In [ ]:
# =========================================
# 7. Descomposición y Comparación Real vs Predicho
# =========================================

def comparar_real_vs_predicho(test, forecast, clave):
    """
    Grafica de manera interactiva los valores reales vs predichos en el conjunto de prueba.

    Parámetros:
    - test (pd.DataFrame): Conjunto de prueba con los valores reales.
    - forecast (pd.DataFrame): DataFrame con las predicciones.
    - clave (str): Clave del modelo (ruta y sentido).
    """
    df_comp = test.merge(forecast[['ds','yhat']], on='ds', how='inner')
    if df_comp.empty:
        logger.warning("No hay datos suficientes para comparar real vs predicho.")
        print("No hay datos suficientes para comparar real vs predicho.")
        return
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_comp['ds'], y=df_comp['y'], mode='lines+markers', name='Real'))
    fig.add_trace(go.Scatter(x=df_comp['ds'], y=df_comp['yhat'], mode='lines+markers', name='Predicho'))
    fig.update_layout(
        title=f"Real vs Predicho - {clave}",
        xaxis_title='Fecha',
        yaxis_title='Demanda',
        hovermode='closest'
    )
    fig.show()



def analizar_modelo(modelos, ruta, sentido, test_data, fecha_prediccion):
    """
    Analiza un modelo específico: compara real vs predicho.

    Parámetros:
    - modelos (dict): Diccionario de modelos entrenados.
    - ruta (str): Nombre de la ruta.
    - sentido (int): Sentido de la ruta (1 o 2).
    - test_data (dict): Diccionario de conjuntos de prueba.
    - fecha_prediccion (str): Fecha a partir de la cual se generaron los pronósticos futuros.
    """
    clave = f"{ruta}_sentido_{sentido}"
    if clave not in modelos:
        print(f"No existe un modelo para {clave}.")
        return

    model = modelos[clave]
    test = test_data.get(clave, None)

    if model and test is not None:
        # Preparar datos de prueba para predicción
        test_prophet = test.rename(columns={'y': 'y'})
        forecast_test = model.predict(test_prophet[['ds', 'es_hora_pico', 'es_fin_de_semana', 'es_dia_festivo']])
        comparar_real_vs_predicho(test_prophet, forecast_test, clave)
    else:
        print(f"Datos insuficientes para el modelo {clave}.")

# Establecer los parámetros que se quieren explorar comparando los valores reales vs predichos
analizar_modelo(modelos, 'PANZENU', 1, test_data,'2024-06-02' )

### 8. Resumen de métricas

En esta última sección, se genera una **tabla interactiva** que resume las principales métricas de los modelos generados o guardados para cada combinación `(ruta, sentido)`:
- **RMSE (Root Mean Squared Error)**: Evalúa la desviación promedio de las predicciones con respecto a la realidad, penalizando los grandes errores.
- **MAE (Mean Absolute Error)**: Mide el promedio de la magnitud de los errores (más sencillo de interpretar).

Estas métricas facilitan la comparación entre varios modelos y ver de un vistazo cuál ofrece mejor desempeño. Un RMSE o MAE más bajo indica un mejor ajuste. Sin embargo, la interpretación también depende del rango de valores de la demanda.

Finalmente, se guardan estas métricas en un archivo JSON para su posterior consulta e histórico de resultados.

---



In [ ]:
# =========================================
# 8. Resumen de Métricas
# =========================================

import plotly.graph_objects as go

def visualizar_metricas_interactivas(df_metricas):
    """
    Crea una tabla interactiva de métricas para RMSE y MAE.

    Parámetros:
    - df_metricas (pd.DataFrame): DataFrame con las métricas.

    Retorna:
    - Figura de la tabla interactiva.
    """

    # Convertir RMSE y MAE a float y redondear a 4 decimales
    df_metricas['RMSE'] = df_metricas['RMSE'].astype(float).round(4)
    df_metricas['MAE'] = df_metricas['MAE'].astype(float).round(4)

    # Construir la tabla sin resaltar ningún valor
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(df_metricas.columns),
            fill_color='paleturquoise',
            align='left'
        ),
        cells=dict(
            # Asegúrate de llamar a las columnas correctas luego de la transformación:
            values=[
                df_metricas['Ruta'],
                df_metricas['Sentido'],
                df_metricas['RMSE'],
                df_metricas['MAE']
            ],
            fill_color='lavender',
            align='left'
        )
    )])

    fig.update_layout(title='Resumen de Métricas de Desempeño')
    fig.show()
    return fig


# Mostrar las métricas
if metricas_actualizadas:
    df_metricas = pd.DataFrame(metricas_actualizadas).T
    df_metricas.reset_index(inplace=True)
    df_metricas[['Ruta','Sentido']] = df_metricas['index'].str.split('_sentido_', expand=True)
    df_metricas = df_metricas[['Ruta', 'Sentido', 'RMSE', 'MAE']].sort_values(by='RMSE', na_position='last')
    visualizar_metricas_interactivas(df_metricas)
else:
    print("No se generaron métricas.")

## 9. Dudas frecuentes

1. **¿Por qué no se ha generado un pronóstico para cierta ruta y sentido?**  
   Es posible que no existan datos históricos suficientes para esa combinación de ruta/sentido o que la sección de entrenamiento se haya saltado debido a la falta de registros. Se recomienda:  
   - Verificar que el archivo CSV incluya la ruta y el sentido buscados.  
   - Si los datos no son uniformes entre rutas, explorar solo modelos para rutas con un set lo suficientemente completo según las recomendaciones y/o aumentar el tamaño del intervalo de predicción.
   - Revisar en el log (`prophet_model.log`) si se reportó algún error o advertencia al entrenar.

2. **¿Qué hacer si aparece un error de `FileNotFoundError` al cargar el CSV?**  
   El cuaderno no encontró el archivo CSV en la ubicación especificada en la variable `ruta_archivo`. Para solucionarlo:  
   - Confirmar la ruta exacta del archivo (incluyendo extensión y carpeta).  
   - Verificar que el archivo exista en Google Drive o en la ruta local indicada.  
   - Asegurarse de haber montado Google Drive correctamente si se trabaja en Google Colab.

3. **¿Por qué los pronósticos aparecen muy bajos o muy altos en horas pico?**  
   - El modelo basa sus predicciones en los datos históricos que dispone. Si la serie histórica no refleja adecuadamente las variaciones en horas pico (por ejemplo, datos incompletos o inconsistentes), el modelo puede subestimar o sobrestimar la demanda.  
   - Ajustar la frecuencia de agregación (`frecuencia`) o agregar más datos históricos pueden mejorar la sensibilidad del modelo en horas pico.

4. **¿Qué hacer si se modifica la variable `split_date` y no se generan resultados en el conjunto de prueba?**  
   - Si `split_date` es muy reciente o muy lejana, puede que no haya datos suficientes para entrenar o para probar el modelo.  
   - Ajustar la fecha de corte a un punto en el que se cuente con datos suficientes en ambas fases (entrenamiento y prueba).  
   - Verificar en el log si se mencionan celdas de entrenamiento vacías o falta de datos para el conjunto de prueba.

5. **¿Por qué el archivo `pronosticos_futuros_consolidados.csv` aparece vacío o con encabezados sin datos?**  
   - Esto sucede si los modelos no generaron predicciones para ninguna ruta (por ejemplo, si hubo un error en la fase de entrenamiento o no existían rutas válidas en el CSV).  
   - Revisar cuidadosamente la sección de entrenamiento y confirmar que, para cada ruta/sentido, se haya completado el proceso y se haya guardado el modelo `.joblib`.

6. **¿Cómo actualizar la ruta base de Google Drive o un entorno local distinto?**  
   - Dentro de la sección de configuración de parámetros, se encuentra la variable que hace referencia a la carpeta base donde se crean los subdirectorios (`modelos_prophet/`). Ajustar esa ruta a la ubicación preferida, tanto para guardar modelos como para almacenar pronósticos y métricas.  
   - Asegurarse de que la ruta exista y de contar con permisos de escritura.
